In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt
import re

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [10]:
todays_games = pd.read_csv('In_Season/Data/todays_bets.csv', index_col = 0)
todays_games

,Away,Home,is_B2B_First_Away,is_B2B_First_Home,is_B2B_Second_Away,is_B2B_Second_Home,Away_Prob_Naive,Home_Prob_Naive,Home_Prob_Adjusted,Away_Prob_Adjusted,...,Away_Odds,Home_Prob_Odds,Away_Prob_Odds,Home_Prob_Diff,Away_Prob_Diff,Home_Bet_Size,Away_Bet_Size,Home_Bet,Away_Bet,Potential_Payoff
0,Philadelphia 76ers,Chicago Bulls,0,1,0,0,0.528106,0.444929,0.484108,0.515892,...,-129.0,0.480769,0.563319,0.003339,-0.047427,0.000643,0.000000,64.302009,0.000000,69.446169
1,Brooklyn Nets,Denver Nuggets,0,0,0,0,0.374566,0.533183,0.760983,0.239017,...,245.0,0.753086,0.289855,0.007897,-0.050838,0.003198,0.000000,319.820050,0.000000,104.859033
2,Detroit Pistons,Minnesota Timberwolves,0,0,0,0,0.239436,0.432112,0.820522,0.179478,...,550.0,0.885057,0.153846,-0.064536,0.025632,0.000000,0.003029,0.000000,302.922247,1666.072357
3,Indiana Pacers,Cleveland Cavaliers,0,0,0,0,0.281105,0.497420,0.831493,0.168507,...,175.0,0.677419,0.363636,0.154074,-0.195130,0.047763,0.000000,4776.293883,0.000000,2274.425659
4,Atlanta Hawks,Dallas Mavericks,0,0,0,0,0.471007,0.476521,0.586414,0.413586,...,107.0,0.563319,0.483092,0.023095,-0.069506,0.005289,0.000000,528.881340,0.000000,409.985535
5,Boston Celtics,Orlando Magic,0,0,0,1,0.551662,0.250972,0.235429,0.764571,...,-480.0,0.217391,0.827586,0.018038,-0.063015,0.002305,0.000000,230.482670,0.000000,829.737612
6,New Orleans Pelicans,Houston Rockets,0,0,0,0,0.348352,0.203080,0.374466,0.625534,...,-195.0,0.384615,0.661017,-0.010150,-0.035483,0.000000,0.000000,0.000000,0.000000,0.000000
7,Milwaukee Bucks,Los Angeles Clippers,0,0,1,0,0.604617,0.399706,0.382212,0.617788,...,-150.0,0.444444,0.600000,-0.062232,0.017788,0.000000,0.004447,0.000000,444.698863,296.465909


In [11]:
todays_games['Potential_Payoff'] = 0
for index, row in todays_games.iterrows():
    if row.Home_Bet > 0:
        if row.Home_Odds < 0:
            todays_games.loc[index, 'Potential_Payoff'] = (row.Home_Bet/abs(row.Home_Odds))*100
        if row.Home_Odds > 0:
            todays_games.loc[index, 'Potential_Payoff'] = row.Home_Bet * (row.Home_Odds/100)
    if row.Away_Bet > 0:
        if row.Away_Odds < 0:
            todays_games.loc[index, 'Potential_Payoff'] = (row.Away_Bet/abs(row.Away_Odds))*100
        if row.Away_Odds > 0:
            todays_games.loc[index, 'Potential_Payoff'] = row.Away_Bet * (row.Away_Odds/100)
todays_games['Date'] = dt.date.today()

In [12]:
todays_games.to_csv('In_Season/Data/todays_bets.csv')